# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,8.39,97,100,11.45,TF,1711762035
1,1,wailua homesteads,22.0669,-159.3780,27.24,71,20,7.20,US,1711762037
2,2,port mathurin,-19.6833,63.4167,26.60,84,97,6.10,MU,1711762038
3,3,khandyga,62.6667,135.6000,-15.03,93,100,1.88,RU,1711762040
4,4,saint-philippe,-21.3585,55.7679,24.77,77,100,1.98,RE,1711762041


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display
# Configure the map plot
map=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    size="Humidity",
    frame_height=550
)
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df=city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)]
wind_df=temp_df.loc[temp_df["Wind Speed"]<4.5]
narrow_df=wind_df.loc[wind_df["Cloudiness"]==0]

# Drop any rows with null values
narrow_df.dropna()

# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,hasaki,35.7333,140.8333,22.38,54,0,2.42,JP,1711762056
79,79,codrington,-38.2667,141.9667,21.95,41,0,3.54,AU,1711762153
113,113,burnet,30.7582,-98.2284,21.02,63,0,3.09,US,1711762067
215,215,marco island,25.9363,-81.7157,22.00,66,0,2.57,US,1711762347
238,238,brisas de zicatela,15.8369,-97.0419,26.29,83,0,4.12,MX,1711762395
272,272,beihai,21.4833,109.1000,24.44,81,0,2.78,CN,1711762450
312,312,cabo san lucas,22.8909,-109.9124,26.17,56,0,3.13,MX,1711762229
361,361,ulladulla,-35.3500,150.4667,23.31,59,0,2.61,AU,1711762590
421,421,abu dhabi,24.4667,54.3667,23.86,69,0,0.51,AE,1711762514


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=narrow_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,hasaki,JP,35.7333,140.8333,54,
79,codrington,AU,-38.2667,141.9667,41,
113,burnet,US,30.7582,-98.2284,63,
215,marco island,US,25.9363,-81.7157,66,
238,brisas de zicatela,MX,15.8369,-97.0419,83,
272,beihai,CN,21.4833,109.1000,81,
312,cabo san lucas,MX,22.8909,-109.9124,56,
361,ulladulla,AU,-35.3500,150.4667,59,
421,abu dhabi,AE,24.4667,54.3667,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,"Lat"]
    longitude=hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    params['categories']=f"accommodation.hotel"


    # Make and API request using the params dictionaty

    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hasaki - nearest hotel: Choshi Plaza Hotel
codrington - nearest hotel: No hotel found
burnet - nearest hotel: Comfort Inn & Suites
marco island - nearest hotel: Marco Island Lakeside Inn
brisas de zicatela - nearest hotel: Casa de Olas
beihai - nearest hotel: Pearl Bay Seaview Hotel
cabo san lucas - nearest hotel: Comfort Rooms


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    size="Humidity",
    frame_height=550,
    hover_cols=["Lng","Lat","City","Humidity","Hotel Name","Country"]
    
    
)
# Display the map
map